Scraping and transform the data into pandas data frame:
---------------------------------------------------------------------------------

All the data for this project will be scrapped from: https://dota2.gamepedia.com/Dota_2_Wiki
which stores all kind of information related to Dota 2.


(1) First of importing all libraries used in this notebook: 

In [4]:
import pandas as pd
from bs4 import BeautifulSoup as besp
import requests as req
import re

(2) Then creating list of all heroes in the game and thier primary attribute. They will be used later for functions and when creating the data frames.

In [5]:
req_1 = req.get(r"https://dota2.gamepedia.com/Table_of_hero_attributes")
soup_obj_1 = besp(req_1.text, 'html.parser')

heroes_lst = [] #list of all hereos inside the game
heroes_Mattribute_lst = [] #list of heroes primary attribute

element_tbody = soup_obj_1.find("tbody") #the component that holdes the table with hereos
element_phero = element_tbody.find_all("tr") #all <tr> tags in the page 

for i in element_phero:
    first_a = i.find("a")
    second_td = i.find_next("td").find_next("td")
    if(first_a != None):
        heroes_lst.append(first_a.get("title"))
    if(second_td != None and first_a != None):
        t_a = second_td.find("a")
        if(t_a != None):
            heroes_Mattribute_lst.append(t_a.get("title"))


print(heroes_lst) #all hereos inside the game
print(heroes_Mattribute_lst) #all heroes primary attributes
print(len(heroes_lst), " == ", len(heroes_Mattribute_lst)) #got everything correct

['Abaddon', 'Alchemist', 'Ancient Apparition', 'Anti-Mage', 'Arc Warden', 'Axe', 'Bane', 'Batrider', 'Beastmaster', 'Bloodseeker', 'Bounty Hunter', 'Brewmaster', 'Bristleback', 'Broodmother', 'Centaur Warrunner', 'Chaos Knight', 'Chen', 'Clinkz', 'Clockwerk', 'Crystal Maiden', 'Dark Seer', 'Dark Willow', 'Dazzle', 'Death Prophet', 'Disruptor', 'Doom', 'Dragon Knight', 'Drow Ranger', 'Earth Spirit', 'Earthshaker', 'Elder Titan', 'Ember Spirit', 'Enchantress', 'Enigma', 'Faceless Void', 'Grimstroke', 'Gyrocopter', 'Hoodwink', 'Huskar', 'Invoker', 'Io', 'Jakiro', 'Juggernaut', 'Keeper of the Light', 'Kunkka', 'Legion Commander', 'Leshrac', 'Lich', 'Lifestealer', 'Lina', 'Lion', 'Lone Druid', 'Luna', 'Lycan', 'Magnus', 'Mars', 'Medusa', 'Meepo', 'Mirana', 'Monkey King', 'Morphling', 'Naga Siren', "Nature's Prophet", 'Necrophos', 'Night Stalker', 'Nyx Assassin', 'Ogre Magi', 'Omniknight', 'Oracle', 'Outworld Destroyer', 'Pangolier', 'Phantom Assassin', 'Phantom Lancer', 'Phoenix', 'Puck', '

(3) Below functions defenition:

In [65]:
def inPage_heroes_extractor(h_lst, soup_obj_page):
    ret_lst = [] #list of heroes in the "version update" page
    heroes_needed = [] #list where the heroes containded in the page
    NBRpattern = r'Nerfed|Buffed|Reworked[ :]' #pattern to detect specific hero changes
    ret_NBR_dict = {"Nerfed": [], "Buffed": [], "Reworked": []} #list containing heroes with certain changes
    NBR_key = "Reworked" #used as flag to switch between: Nerfed/Buffed/Reworked

    elmt_meta = soup_obj_page.find_all("meta")
    for i in elmt_meta:
        if(i.get("property") == "og:description"):
            heroes_needed = i.get("content").split(" ") #gets the heroes from side window.
 
    rrr = []
    k = 0
    while(k < len(heroes_needed)):
        Kstep_control = False
        #N,B,R (nerfed/buffed/reworked)
        
        NBR_text = re.search(NBRpattern, heroes_needed[k])
        if(NBR_text):
            if(NBR_text.group(0) == "Nerfed"):
                NBR_key = "Nerfed"
            if(NBR_text.group(0) == "Buffed"):
                NBR_key = "Buffed"
            if(NBR_text.group(0) == "Reworked"):
                NBR_key = "Reworked"
        #
        #
        #
        for j in h_lst:
            j_skip = j.split(" ")
            #
            if(k >= len(heroes_needed)):
                break
            #
            if(j_skip[0] == heroes_needed[k]): #the hero matches from heroes list
              
                if(heroes_needed[k] != "Shadow" and heroes_needed[k] != "Phantom"): #regular else at the end added twice special cases
                    rrr.append(j)
                    ret_NBR_dict[NBR_key].append(j)
                if((k+1) < len(heroes_needed)):  #prevent out of bound  
                    if(heroes_needed[k] == "Shadow"):
                        if(heroes_needed[k+1] == "Demon"):
                            rrr.append("Shadow Demon")
                            ret_NBR_dict[NBR_key].append("Shadow Demon")

                        if(heroes_needed[k+1] == "Fiend"):
                            rrr.append("Shadow Fiend")
                            ret_NBR_dict[NBR_key].append("Shadow Fiend")

                        if(heroes_needed[k+1] == "Shaman"):
                            rrr.append("Shadow Shaman")
                            ret_NBR_dict[NBR_key].append("Shadow Shaman")

                    if(heroes_needed[k] == "Phantom"):
                        if(heroes_needed[k+1] == "Lancer"):
                            rrr.append("Phantom Lancer")
                            ret_NBR_dict[NBR_key].append("Phantom Lancer")

                        if(heroes_needed[k+1] == "Assassin"):
                            rrr.append("Phantom Assassin")
                            ret_NBR_dict[NBR_key].append("Phantom Assassin")
                        

                k += len(j_skip)
                #CHECK FOR NBR at -1
                
                if((k-1) < len(heroes_needed)): #prevent out of bound
                    NBR_text = re.search(NBRpattern, heroes_needed[k-1])
                    if(NBR_text):
                        if(NBR_text.group(0) == "Nerfed"):
                            NBR_key = "Nerfed"
                        if(NBR_text.group(0) == "Buffed"):
                            NBR_key = "Buffed"
                        if(NBR_text.group(0) == "Reworked"):
                            NBR_key = "Reworked"
                #NBR CHECK END
                Kstep_control = True #fixes problem where jump twice
        if(Kstep_control == False):
            k += 1 #if no hero matched go to next

    return rrr, ret_NBR_dict #returns list with heroes in page and dict of which of them BNR

In [7]:
def inPage_heroChange_Extractor(hero_111, soup_obj_page):
    elmt_div = soup_obj_page.find_all("div") #all <div> tags in the page 
    ret_change_lst = [] #list with hero changes
    
    for i in elmt_div:
        a_in_div = i.find("a")
        if(a_in_div != None):
            if(hero_111 == a_in_div.text):
                all_li = i.find_next("ul").find_all("li")
                for ll in all_li:
                    ret_change_lst.append(ll.text)
                    
    return ret_change_lst

In [8]:
def get_hero_abilities(soupobj):
    pattern_ability = r'(\w+[\w+\W+]\w+ \w*?)( *)[Link DV QT]'
    
    ret_lst = []
    
    needed_div = soupobj.find_all("div", "ability-background")
    for ttt in needed_div:
        pls = ttt.find_all("div")

        only_ability_name = re.findall(pattern_ability, pls[1].text)

        if(only_ability_name):
            ret_lst.append(only_ability_name[0][0])
    return ret_lst

In [9]:
def get_all_heroes_abilities(soup_page):
    element_tbody = soup_page.find("tbody") #the component that holdes the table
    element_phero = element_tbody.find_all("tr") #tr tags 
    
    custom_col = ["Heroes", "Abilities"]
    ret_dataFrame = pd.DataFrame(columns=custom_col)
    
    for i in element_phero:
        first_a = i.find("a")
        if(first_a != None):
            hero_name = first_a.get("title") #heroes name
            hero_url = "https://dota2.gamepedia.com" + first_a.get("href") #single hero url
            #
            rRr = req.get(hero_url)
            soup_super_obj = besp(rRr.text, 'html.parser')
            #
            ret_dataFrame = ret_dataFrame.append({"Heroes": hero_name, "Abilities": get_hero_abilities(soup_super_obj)}, ignore_index=True)
    
    return ret_dataFrame

(4) Now using 2 functions above we create data frame with heroes and thier abilities, it will be used later in other function.

In [10]:
df_heroAbilities = get_all_heroes_abilities(soup_obj_1) 
print(df_heroAbilities.head())
df_heroAbilities.to_csv("Heroes_Abilities.csv", encoding='utf-8', index=False)

               Heroes                                          Abilities
0             Abaddon  [Mist Coil , Aphotic Shield , Curse of Aver, B...
1           Alchemist  [Acid Spray , Unstable Concoction , Unstable C...
2  Ancient Apparition  [Cold Feet , Ice Vortex , Chilling Touch , Ice...
3           Anti-Mage  [Mana Break , Blink , Counterspell , Blink Fra...
4          Arc Warden  [Flux , Magnetic Field , Spark Wraith , Tempes...


(5) Now last part, next 2 functions will combine everything. They will make for each "The International" a data frame with the changes made to heroes, from  "The International" till the next one.

In [11]:
def text_to_numeric(hero_a, data_text):
    #other initialize parameters
    file_path = r"path\Heroes_Abilities.csv"
    heroes_abilities_df = pd.read_csv(file_path, converters={'Abilities': eval}) #makes abilities when read as list and not str
    #columns: Heroes | Abilities
    right_hero_fdf = heroes_abilities_df[heroes_abilities_df["Heroes"] == hero_a] #hero row from df
    right_hero_abilities_lst = right_hero_fdf.iloc[0, 1] #list of heroes abilities
    ret_ckey = "asd"
    ret_num = 0
    
    #list of patterns to check
    pattern = r'[A a]gility'
    in_pattern_ability = r"[ *]"
    in_pattern_stats = r"[ *]Strength|Agility|Intelligence|Base Attack|Base damage|Movement speed|Base movement[ *]"
    in_pattern_plus = r"[ *]increased[ *]"
    in_pattern_minus = r"[ *]reduced[ *]"
    
    #universal pattern
    #ability pattern
    for i in right_hero_abilities_lst:
        in_pattern_ability += (i[:-1] + "|")
    in_pattern_ability = (in_pattern_ability[:-1] + "[ *]")
    #
    
    #text with hero change that matches pattern
    extracted_text = re.findall(pattern, data_text)
    has_ability_change_text = re.search(in_pattern_ability, data_text)
    has_stats_change_text = re.search(in_pattern_stats, data_text)
    
    #transform and return numeric data, with column to change
    if(has_ability_change_text):
        ret_ckey = "Abbility_changed"
        ret_num = 1
        return ret_ckey, ret_num
        
    if(has_stats_change_text):
        has_plus = re.search(in_pattern_plus, data_text)
        has_minus = re.search(in_pattern_minus, data_text)
        
        if(has_plus):
            ret_ckey = "Stat_plus"
            ret_num = 1
            return ret_ckey, ret_num
    
        if(has_minus):
            ret_ckey = "Stat_minus"
            ret_num = 1
            return ret_ckey, ret_num
        
        if(has_plus == None and has_minus == None):
            ret_ckey = "Other_change"
            ret_num = 1
            return ret_ckey, ret_num
    return None, 999

In [66]:
#data frame Block with changes from a year, from end of one TI till next TI.
def Create_TI_df(versions_lst):
    custom_c = ["Name", "Primary_attribute", "Stat_plus", "Stat_minus", 
                 "Abbility_changed", "Abbility_rework", 
                 "Buffed", "Nerfed", "Reworked", "Other_change", 
                 "In_TI" ]
    Block_df = pd.DataFrame(columns=custom_c)
    Block_df["Name"] = heroes_lst
    Block_df.fillna(0, inplace=True)

    #Make loop to process list of version pages
    for zzz in versions_lst:
        #Soup-obj
        fi_r = req.get(zzz)
        bt_soup_obj = besp(fi_r.text, 'html.parser')
        #
        #
        bt_heroes_in_page_lst = []
        bt_heroes_in_page_BNR_dict = []
        bt_specific_hero_changeTxt_lst = []
        c_key = "None"
        num = 99999
        BNR_key = "None"

        bt_heroes_in_page_lst, bt_heroes_in_page_BNR_dict = inPage_heroes_extractor(heroes_lst, bt_soup_obj) #returns: all heroes in page NOT sorted
                                                                                            #and list of BNR.

        bt_sorted_heroes = sorted(bt_heroes_in_page_lst)#sort so be in page order    
        for iii in bt_sorted_heroes:

            bt_specific_hero_changeTxt_lst = inPage_heroChange_Extractor(iii, bt_soup_obj) #returns: hero change text, list, in page
            for jjj in bt_specific_hero_changeTxt_lst:
                c_key, num = text_to_numeric(iii, jjj) 
                #check: NBR, c_key. determine what and how changes
                if any(iii in ttt for ttt in bt_heroes_in_page_BNR_dict["Nerfed"]):
                    BNR_key = "Nerfed"

                if any(iii in ttt for ttt in bt_heroes_in_page_BNR_dict["Buffed"]):
                    BNR_key = "Buffed"

                if any(iii in ttt for ttt in bt_heroes_in_page_BNR_dict["Reworked"]):
                    BNR_key = "Reworked"

                #print("Hero: ", iii, " Column: ", c_key, " Value: ", num, " NBR: ", BNR_key)

                if(BNR_key == "Reworked"): #Reworked means the hero changed drasticly
                    Block_df.loc[Block_df.Name == iii, "Reworked"] += 1

                if(BNR_key == "Nerfed" or BNR_key == "Buffed"):
                    if(c_key != None):
                        Block_df.loc[Block_df.Name == iii, c_key] += num
                        
                    if(BNR_key == "Nerfed"):
                        Block_df.loc[Block_df.Name == iii, BNR_key] += 1
                        
                    if(BNR_key == "Buffed"):
                        Block_df.loc[Block_df.Name == iii, BNR_key] += 1

            
    return Block_df #return df at the end. each such df represent one year of changes.

(6) Creating data frames from 2013-2019, 2011 not included because was first tournament so was new. And in 2020 there was no The International because of Covid-19. 

In [68]:
ver_url = r"https://dota2.gamepedia.com/Game_Versions"
req_ver  = req.get(ver_url)
ver_soup_obj = besp(req_ver.text, 'html.parser')

TI_version_lst = []

ll_ex = ver_soup_obj.find_all("a", "mw-redirect")
for i in ll_ex:
    v_url = "https://dota2.gamepedia.com/Version_" + i.text
    if(v_url[36] == str(7) or v_url[36] == str(6)):
        TI_version_lst.append(v_url)

dft = Create_TI_df(TI_version_lst)

print(dft.head())

#insert In_TI values, 1 means the hero was chosen during TI match
hereos_chosen_in_TI = ["Abaddon", "Ancient Apparition", "Anti-Mage", "Axe",
                       "Batrider", "Brewmaster", "Chaos Knight", "Crystal Maiden", "Dark Seer",
                       "Doom", "Earthshaker", "Elder Titan", "Ember Spirit", "Gyrocopter", "Io", 
                       "Juggernaut", "Keeper of the Light", "Lich", "Magnus", "Naga Siren", "Nature's Prophet",
                       "Necrophos", "Puck", "Rubick", "Shadow Shaman", "Storm Spirit", "Templar Assassin", 
                       "Tidehunter", "Timbersaw", "Tiny", "Zeus"]

for i in hereos_chosen_in_TI:
    dft.loc[dft.Name == i, "In_TI"] = 1
#
#inserting heroes attributes from "heroes_Mattribute_lst"
dft["Primary_attribute"] = heroes_Mattribute_lst
#

dft.to_csv("TI_df.csv", encoding='utf-8', index=False)

                 Name  Primary_attribute  Stat_plus  Stat_minus  \
0             Abaddon                  0          0           2   
1           Alchemist                  0          2           1   
2  Ancient Apparition                  0          1           0   
3           Anti-Mage                  0          1           4   
4          Arc Warden                  0          3           1   

   Abbility_changed  Abbility_rework  Buffed  Nerfed  Reworked  Other_change  \
0                21                0      23      16         0             1   
1                44                0      44      27         0             0   
2                26                0      25       9         0             0   
3                15                0       7      24         0             1   
4                37                0      33      27         3             1   

   In_TI  
0      0  
1      0  
2      0  
3      0  
4      0  
